# Attentional Networks in Computer Vision
Prepared by Comp411 Teaching Unit (TA Can Küçüksözen) in the context of Computer Vision with Deep Learning Course. Do not hesitate to ask in case you have any questions, contact me at: ckucuksozen19@ku.edu.tr

Up until this point, we have worked with deep fully-connected networks, convolutional networks and recurrent networks using them to explore different optimization strategies and network architectures. Fully-connected networks are a good testbed for experimentation because they are very computationally efficient, on the other hand, most successful image processing methods use convolutional networks. However recent state-of-the-art results on computer vision realm are acquired using Attentional layers and Transformer architectures.

First you will implement several layer types that are used in fully attentional networks. You will then use these layers to train an Attentional Image Classification network, specifically a smaller version of Vision Transformer (VIT) on the CIFAR-10 dataset. The original paper can be accessed via the following link: https://arxiv.org/pdf/2010.11929.pdf

# Part I. Preparation

First, we load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In previous parts of the assignment we had to write our own code to download the CIFAR-10 dataset, preprocess it, and iterate through it in minibatches; PyTorch provides convenient tools to automate this process for us.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [2]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./comp411/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./comp411/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./comp411/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


You have an option to **use GPU by setting the flag to True below**. It is not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment. 

In [3]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


# Part II. Barebones Transformers: Self-Attentional Layer

Here you will complete the implementation of the Pytorch nn.module `SelfAttention`, which will perform the forward pass of a self-attentional layer. Our implementation of the SelfAttentional layer will include three distinct fully connected layers which will be responsible of:

1. A fully connected layer, `W_Q`, which will be used to project our input into `queries`
2. A fully connected layer, `W_K`, which will be used to project our input into `keys`
3. A fully connected layer, `W_V`, which will be used to project our input into `values`

After defining such three fully connected layers, and obtain our `queries, keys, and values` variables at the beginning of our forward pass, the following operations should be carried out in order to complete the attentional layer implementation.

1. Seperate each of `query, key, and value` projections into their respective heads. In other words, split the feature vector dimension of each matrix into necessarry number of chunks.

2. Compute the `Attention Scores` between each pair of sequence elements via conducting a scaled dot product operation between every pair of `queries` and `keys`. Note that `Attention Scores` matrix should have the size of `[# of queries , # of keys]`

3. Calculate the `Attention Weights` of each query by applying the non-linear `Softmax` normalization accross the `keys` dimension of the `Attention Scores` matrix.

4. Obtain the output combination of `values` by matrix multiplying `Attention Weights` with `values`

5. Reassemble heads into one flat vector and return the output.

**HINT**: For a more detailed explanation of the self attentional layer, examine the Appendix A of the original ViT manuscript here:  https://arxiv.org/pdf/2010.11929.pdf 

In [5]:
class SelfAttention(nn.Module):
    
    def __init__(self, input_dims, head_dims=128, num_heads=2,  bias=False):
        super(SelfAttention, self).__init__()
        
        ## initialize module's instance variables
        self.input_dims = input_dims
        self.head_dims = head_dims
        self.num_heads = num_heads
        self.proj_dims = head_dims * num_heads
        
        ## Declare module's parameters
        self.W_Q = nn.Linear(input_dims, self.proj_dims,bias=bias)
        self.W_K = nn.Linear(input_dims, self.proj_dims,bias=bias)
        self.W_V = nn.Linear(input_dims, self.proj_dims,bias=bias)
        
        self.W_O = nn.Linear(self.proj_dims,self.proj_dims,bias=bias)

        self.init_weights()
        
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.1)

    def forward(self, x):
        ## Input of shape, [B, N, D] where:
        ## - B denotes the batch size
        ## - N denotes number of sequence elements. I.e. the number of patches + the class token 
        ## - D corresponds to model dimensionality
        b,n,d = x.shape
        
        ## Construct queries,keys,values
        q_ = self.W_Q(x)
        k_ = self.W_K(x)
        v_ = self.W_V(x)
        
#         print(f"q_.shape: {q_.shape}")
#         print(f"k_.shape: {k_.shape}")
#         print(f"v_.shape: {v_.shape}")
        
        ## Seperate q,k,v into their corresponding heads,
        ## After this operation each q,k,v will have the shape: [B,H,N,D//H] where
        ## - B denotes the batch size
        ## - H denotes number of heads
        ## - N denotes number of sequence elements. I.e. the number of patches + the class token 
        ## - D//H corresponds to per head dimensionality
        q, k, v = map(lambda z: torch.reshape(z, (b, n, self.num_heads, self.head_dims)).permute(0,2,1,3), [q_,k_,v_])
       
#         print(f"q.shape: {q.shape}")
#         print(f"k.shape: {k.shape}")
#         print(f"v.shape: {v.shape}")
    
        attn_out = None
        
        #########################################################################################
        # TODO: Complete the forward pass of the SelfAttention layer, follow the comments below #
        #########################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        
        ## Compute attention logits. Note that this operation is implemented as a
        ## batched matrix multiplication between q and k, the output is scaled by 1/(D//H)^(1/2)
        ## inputs are queries and keys that are both of size [B,H,N,D//H]
        ## Output Attention logits should have the size: [B,H,N,N]
        
        
        attn_out = torch.matmul(q, k.transpose(-2, -1)) * (1 / (self.head_dims ** 0.5))
#         print(f"attn_out.shape: {attn_out.shape}")
        
        ## Compute attention Weights. Recall that this operation is conducted as a
        ## Softmax Normalization across the keys dimension. 
        ## Hint: You can apply the Softmax operation across the final dimension
        
        attn_out = attn_out.softmax(dim=-1)
#         print(f"attn_out.shape: {attn_out.shape}")
    
        ## Compute attention output values. Bear in mind that this operation is applied as a 
        ## batched matrix multiplication between the Attention Weights matrix and 
        ## the values tensor. After computing output values, the output should be reshaped
        ## Inputs are Attention Weights with size [B, H, N, N], values with size [B, H, N, D//H(64)] -> [B, H, N, D]
        ## Output should be of size [B, N, D]
        ## Hint: you should use torch.matmul, torch.permute, torch.reshape in that order
        ##       (or any other equivalent torch operations)
        
        attn_out = torch.matmul(attn_out, v).transpose(1, 2).reshape(b, n, self.proj_dims)
#         print(f"attn_out.shape: {attn_out.shape}")
        
        ## Compute output feature map. This operation is just passing the concatenated attention 
        ## output that we have just obtained through a final projection layer W_O.
        ## Both the input and the output should be of size [B, N, self.proj_dims]
        
        attn_out = self.W_O(attn_out)
        
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             
        ################################################################################
        
        return attn_out

After defining the forward pass of the Self-Attentional Layer above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 64).

In [6]:
def test_self_attn_layer():
    x = torch.zeros((64, 16, 32), dtype=dtype)  # minibatch size 64, sequence elements size 16, feature channels size 32
    layer = SelfAttention(input_dims=32, head_dims=64, num_heads=4)
    out = layer(x)
    print(out.size())  # you should see [64,16,256]
    
test_self_attn_layer()

torch.Size([64, 16, 256])


# Part III. Barebones Transformers: Transformer Encoder Block

Here you will complete the implementation of the Pytorch nn.module `TransformerBlock`, which will perform the forward pass of a Transfomer Encoder Block. You can refer to Figure 1 of the original manuscript of ViT from this link: https://arxiv.org/pdf/2010.11929.pdf in order to get yourself familiar with the architecture.



In [7]:
## Implementation of a two layer GELU activated Fully Connected Network is provided for you below:

class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims, bias=True):
        super().__init__()
        
        self.fc_1 = nn.Linear(input_dims, hidden_dims, bias=bias)
        self.fc_2 = nn.Linear(hidden_dims, output_dims, bias=bias)
        
        self.init_weights()
        
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.1)
        
    def forward(self, x):
        o = F.gelu(self.fc_1(x))
        o = self.fc_2(o)
        return o

In [8]:
## Build from scratch a TransformerBlock Module. Note that the architecture of this
## module follows a simple computational pipeline:
##                                               x------------------------------------------->x
## input --> layernorm --> SelfAttention --> skip connection --> layernorm --> MLP ---> skip connection ---> output
## Note that the TransformerBlock module works on a single hidden dimension hidden_dims,
## in order to faciliate skip connections with ease. Be careful about the input arguments
## to the SelfAttention block.


class TransformerBlock(nn.Module):
    def __init__(self, hidden_dims, num_heads=4, bias=False):
        super(TransformerBlock, self).__init__()
        
        ###############################################################
        # TODO: Complete the consturctor of  TransformerBlock module  #
        ###############################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)****

        self.layer_norm1 = nn.LayerNorm(hidden_dims)
        self.attention   = SelfAttention(input_dims=hidden_dims, head_dims=hidden_dims//num_heads,
                                         num_heads=num_heads)
        self.layer_norm2 = nn.LayerNorm(hidden_dims)
        self.mlp         = nn.Linear(in_features=hidden_dims, out_features=hidden_dims)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ###################################################################
        #                                 END OF YOUR CODE                #             
        ###################################################################

    def forward(self, x):
        
        ##############################################################
        # TODO: Complete the forward of TransformerBlock module      #
        ##############################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)****
        
        out1 = self.layer_norm1(x)
        out2 = self.attention(out1)
        out3 = self.layer_norm2(out2 + x) 
        out4 = self.mlp(out3)
        
        return out4 + x
        
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ###################################################################
        #                                 END OF YOUR CODE                #             
        ###################################################################

After defining the forward pass of the Transformer Block Layer above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 64).

In [9]:
def test_transfomerblock_layer():
    x = torch.zeros((64, 16, 128), dtype=dtype)  # minibatch size 64, sequence elements size 16, feature channels size 128
    layer = TransformerBlock(128, 4) # hidden dims size 128, heads size 4
    out = layer(x)
    print(out.size())  # you should see [64,16,128]
test_transfomerblock_layer()

torch.Size([64, 16, 128])


# Part IV The Vision Transformer (ViT)

The final implementation for the Pytorch nn.module `ViT` is given to you below, which will perform the forward pass of the Vision Transformer. Study it and get yourself familiar with the API.


In [10]:
class ViT(nn.Module):
    def __init__(self, hidden_dims, input_dims=3, output_dims=10, num_trans_layers = 4, num_heads=4, image_k=32, patch_k=4, bias=False):
        super(ViT, self).__init__()
                
        ## initialize module's instance variables
        self.hidden_dims = hidden_dims
        self.input_dims = input_dims
        self.output_dims = output_dims
        self.num_trans_layers = num_trans_layers
        self.num_heads = num_heads
        self.image_k = image_k
        self.patch_k = patch_k
        
        self.image_height = self.image_width = image_k
        self.patch_height = self.patch_width = patch_k
        
        assert self.image_height % self.patch_height == 0 and self.image_width % self.patch_width == 0,\
                'Image size must be divisible by the patch size.'

        self.num_patches = (self.image_height // self.patch_height) * (self.image_width // self.patch_width)
        self.patch_flat_len = self.patch_height * self.patch_width
        
        ## Declare module's parameters
        
        ## ViT's flattened patch embedding projection:
        self.linear_embed = nn.Linear(self.input_dims*self.patch_flat_len, self.hidden_dims)
        
        ## Learnable positional embeddings, an embedding is learned for each patch location and the class token
        self.pos_embedding = nn.Parameter(torch.randn(1, self.num_patches + 1, self.hidden_dims))
        
        ## Learnable classt token and its index among attention sequence elements.
        self.cls_token = nn.Parameter(torch.randn(1,1,self.hidden_dims))
        self.cls_index = torch.LongTensor([0])
        
        ## Declare cascaded Transformer blocks:
        transformer_encoder_list = []
        for _ in range(self.num_trans_layers):
            transformer_encoder_list.append(TransformerBlock(self.hidden_dims, self.num_heads, bias))
        self.transformer_encoder = nn.Sequential(*transformer_encoder_list)
        
        ## Declare the output mlp:
        self.out_mlp = MLP(self.hidden_dims, self.hidden_dims, self.output_dims)
         
    def unfold(self, x, f = 7, st = 4, p = 0):
        ## Create sliding window pathes using nn.Functional.unfold
        ## Input dimensions: [B,D,H,W] where
        ## --B : input batch size
        ## --D : input channels
        ## --H, W: input height and width
        ## Output dimensions: [B,N,H*W,D]
        ## --N : number of patches, decided according to sliding window kernel size (f),
        ##      sliding window stride and padding.
        b,d,h,w = x.shape
        x_unf = F.unfold(x, (f,f), stride=st, padding=p)    
        x_unf = torch.reshape(x_unf.permute(0,2,1), (b,-1,d,f*f)).transpose(-1,-2)
        n = x_unf.size(1)
        return x_unf,n
    
    def forward(self, x):
        b = x.size(0)
        ## create sliding window patches from the input image
        x_patches,n = self.unfold(x, self.patch_height, self.patch_height, 0)
        ## flatten each patch into a 1d vector: i.e. 3x4x4 image patch turned into 1x1x48
        x_patch_flat = torch.reshape(x_patches, (b,n,-1))
        ## linearly embed each flattened patch
        x_embed = self.linear_embed(x_patch_flat)
        
        ## retrieve class token 
        cls_tokens = self.cls_token.repeat(b,1,1)
        ## concatanate class token to input patches
        xcls_embed = torch.cat([cls_tokens, x_embed], dim=-2)
        
        ## add positional embedding to input patches + class token 
        xcls_pos_embed = xcls_embed + self.pos_embedding
        
        ## pass through the transformer encoder
        trans_out = self.transformer_encoder(xcls_pos_embed)
        
        ## select the class token 
        out_cls_token = torch.index_select(trans_out, -2, self.cls_index.to(trans_out.device))
        
        ## create output
        out = self.out_mlp(out_cls_token)
        
        return out.squeeze(-2)

After defining the forward pass of the ViT above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 64).

In [11]:
def test_vit():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size 3,32,32
    model = ViT(hidden_dims=128, input_dims=3, output_dims=10, num_trans_layers = 4, num_heads=4, image_k=32, patch_k=4)
    out = model(x)
    print(out.size())  # you should see [64,10]
test_vit()

torch.Size([64, 10])


# Part V. Train the ViT

### Check Accuracy
Given any minibatch of input data and desired targets, we can check the classification accuracy of a neural network. 

The check_batch_accuracy function is provided for you below:

In [12]:
def check_batch_accuracy(out, target,eps=1e-7):
    b, c = out.shape
    with torch.no_grad():
        _, pred = out.max(-1) 
        correct = np.sum(np.equal(pred.cpu().numpy(), target.cpu().numpy()))
    return correct, np.float(correct) / (b)

### Training Loop
As we have already seen in the Second Assignment, in our PyTorch based training loops, we use an Optimizer object from the `torch.optim` package, which abstract the notion of an optimization algorithm and provides implementations of most of the algorithms commonly used to optimize neural networks.

In [13]:
def train(network, optimizer, trainloader):
    """
    Train a model on CIFAR-10 using the PyTorch Module API for a single epoch
    
    Inputs:
    - network: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - trainloader: Iterable DataLoader object that fetches the minibatches
    
    Returns: overall training accuracy for the epoch
    """
    print('\nEpoch: %d' % epoch)
    network.train()  # put model to training mode
    network = network.to(device=device)  # move the model parameters to CPU/GPU
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = Variable(inputs.to(device)), targets.to(device)  # move to device, e.g. GPU
            
        outputs = network(inputs)
        loss =  F.cross_entropy(outputs, targets)
            
        # Zero out all of the gradients for the variables which the optimizer
        # will update.
        optimizer.zero_grad() 

        # This is the backwards pass: compute the gradient of the loss with
        # respect to each  parameter of the model.
        loss.backward()
            
        # Actually update the parameters of the model using the gradients
        # computed by the backwards pass.
        optimizer.step()
            
        loss = loss.detach()
        train_loss += loss.item()
        correct_p, _ = check_batch_accuracy(outputs, targets) 
        correct += correct_p
        total += targets.size(0)

        print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
        % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
    return 100.*correct/total

### Evaluation Loop
We have also prepared a Evaluation loop in order to determine our networks capabilities in terms of classification accuracy on a given dataset, either the training, or the validation split

In [14]:
def evaluate(network, evalloader):
    """
    Evaluate a model on CIFAR-10 using the PyTorch Module API for a single epoch
    
    Inputs:
    - network: A PyTorch Module giving the model to train.
    - evalloader: Iterable DataLoader object that fetches the minibatches
    
    Returns: overall evaluation accuracy for the epoch
    """
    network.eval() # put model to evaluation mode
    network = network.to(device=device)  # move the model parameters to CPU/GPU
    eval_loss = 0
    correct = 0
    total = 0
    print('\n---- Evaluation in process ----')
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(evalloader):
            inputs, targets = inputs.to(device), targets.to(device) # move to device, e.g. GPU
            outputs = network(inputs)
            loss = F.cross_entropy(outputs, targets)
            
            eval_loss += loss.item()
            correct_p, _ = check_batch_accuracy(outputs, targets)
            correct += correct_p
            total += targets.size(0)
            print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (eval_loss/(batch_idx+1), 100.*correct/total, correct, total))
    return 100.*correct/total

### Overfit a ViT
Now we are ready to run the training loop. A nice trick is to train your model with just a few training samples in order to see if your implementation is actually bug free. 

Simply pass the input size, hidden layer size, and number of classes (i.e. output size) to the constructor of `ViT`. 

You also need to define an optimizer that tracks all the learnable parameters inside `ViT`. We prefer to use `Adam` optimizer for this part.

You should be able to overfit small datasets, which will result in very high training accuracy and comparatively low validation accuracy.

In [15]:
sample_idx_tr = torch.randperm(len(cifar10_train))[:100]
sample_idx_val = torch.randperm(len(cifar10_train))[-100:]

trainset_sub = torch.utils.data.Subset(cifar10_train, sample_idx_tr)
valset_sub = torch.utils.data.Subset(cifar10_train, sample_idx_val)

print("For overfitting experiments, the subset of the dataset that is used has {} sample images".format(len(trainset_sub)))

batch_size_sub = 25
trainloader_sub = torch.utils.data.DataLoader(trainset_sub, batch_size=batch_size_sub, shuffle=True)
valloader_sub = torch.utils.data.DataLoader(valset_sub, batch_size=batch_size_sub, shuffle=False)

print('==> Data ready, batchsize = {}'.format(batch_size_sub))

For overfitting experiments, the subset of the dataset that is used has 100 sample images
==> Data ready, batchsize = 25


In [16]:
learning_rate = 0.001
input_dims = 3
hidden_dims = 128
output_dims=10
num_trans_layers = 4
num_heads=4
image_k=32
patch_k=4

network = None
optimizer = None

################################################################################
# TODO: Instantiate your ViT model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

network = ViT(hidden_dims=hidden_dims, input_dims=input_dims,
            output_dims=output_dims, num_trans_layers = num_trans_layers, 
            num_heads=num_heads, image_k=image_k, patch_k=patch_k)

optimizer = optim.Adam(network.parameters(), lr=learning_rate)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

tr_accs=[]
eval_accs=[]
for epoch in range(15):
    tr_acc = train(network, optimizer, trainloader_sub)
    print('Epoch {} of training is completed, Training accuracy for this epoch is {}'\
              .format(epoch, tr_acc))  
    
    eval_acc = evaluate(network, valloader_sub)
    print('Evaluation of Epoch {} is completed, Validation accuracy for this epoch is {}'\
              .format(epoch, eval_acc))  
    tr_accs.append(tr_acc)
    eval_accs.append(eval_acc)
    
print("\nFinal train set accuracy is {}".format(tr_accs[-1]))
print("Final val set accuracy is {}".format(eval_accs[-1]))


Epoch: 0
Loss: 3.524 | Acc: 4.000% (1/25)


/Users/firattamur/opt/anaconda3/envs/comp411-v3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


Loss: 3.384 | Acc: 8.000% (4/50)
Loss: 3.147 | Acc: 6.667% (5/75)
Loss: 3.049 | Acc: 7.000% (7/100)
Epoch 0 of training is completed, Training accuracy for this epoch is 7.0

---- Evaluation in process ----
Loss: 2.828 | Acc: 12.000% (3/25)
Loss: 2.600 | Acc: 16.000% (8/50)
Loss: 2.779 | Acc: 14.667% (11/75)
Loss: 2.703 | Acc: 15.000% (15/100)
Evaluation of Epoch 0 is completed, Validation accuracy for this epoch is 15.0

Epoch: 1
Loss: 2.481 | Acc: 16.000% (4/25)
Loss: 2.345 | Acc: 14.000% (7/50)
Loss: 2.407 | Acc: 12.000% (9/75)
Loss: 2.366 | Acc: 16.000% (16/100)
Epoch 1 of training is completed, Training accuracy for this epoch is 16.0

---- Evaluation in process ----
Loss: 2.259 | Acc: 32.000% (8/25)
Loss: 2.147 | Acc: 28.000% (14/50)
Loss: 2.208 | Acc: 21.333% (16/75)
Loss: 2.224 | Acc: 20.000% (20/100)
Evaluation of Epoch 1 is completed, Validation accuracy for this epoch is 20.0

Epoch: 2
Loss: 1.972 | Acc: 40.000% (10/25)
Loss: 2.039 | Acc: 30.000% (15/50)
Loss: 2.072 | Acc: 2

## Train the net
By training the four-layer ViT network for three epochs, with untuned hyperparameters that are initialized as below,  you should achieve greater than 50% accuracy both on the training set and the test set:

In [17]:
learning_rate = 0.001
input_dims = 3
hidden_dims = 128
output_dims=10
num_trans_layers = 4
num_heads=4
image_k=32
patch_k=4

network = None
optimizer = None

################################################################################
# TODO: Instantiate your ViT model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    
network = ViT(hidden_dims=hidden_dims, input_dims=input_dims,
            output_dims=output_dims, num_trans_layers = num_trans_layers, 
            num_heads=num_heads, image_k=image_k, patch_k=patch_k)

optimizer = optim.Adam(network.parameters(), lr=learning_rate)
# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

tr_accs=[]
test_accs=[]
for epoch in range(3):
    tr_acc = train(network, optimizer, loader_train)
    print('Epoch {} of training is completed, Training accuracy for this epoch is {}'\
              .format(epoch, tr_acc))  
    
    test_acc = evaluate(network, loader_test)
    print('Evaluation of Epoch {} is completed, Test accuracy for this epoch is {}'\
              .format(epoch, test_acc))  
    
    tr_accs.append(tr_acc)
    test_accs.append(test_acc)
    
print("\nFinal train set accuracy is {}".format(tr_accs[-1]))
print("Final test set accuracy is {}".format(test_accs[-1]))


Epoch: 0


/Users/firattamur/opt/anaconda3/envs/comp411-v3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


Loss: 3.094 | Acc: 6.250% (4/64)
Loss: 2.875 | Acc: 7.812% (10/128)
Loss: 2.750 | Acc: 9.896% (19/192)
Loss: 2.673 | Acc: 11.719% (30/256)
Loss: 2.628 | Acc: 12.500% (40/320)
Loss: 2.559 | Acc: 13.281% (51/384)
Loss: 2.505 | Acc: 14.062% (63/448)
Loss: 2.471 | Acc: 14.648% (75/512)
Loss: 2.467 | Acc: 14.062% (81/576)
Loss: 2.441 | Acc: 15.156% (97/640)
Loss: 2.414 | Acc: 15.625% (110/704)
Loss: 2.396 | Acc: 15.495% (119/768)
Loss: 2.369 | Acc: 15.986% (133/832)
Loss: 2.354 | Acc: 16.406% (147/896)
Loss: 2.341 | Acc: 17.188% (165/960)
Loss: 2.326 | Acc: 17.578% (180/1024)
Loss: 2.311 | Acc: 18.290% (199/1088)
Loss: 2.304 | Acc: 18.316% (211/1152)
Loss: 2.295 | Acc: 18.668% (227/1216)
Loss: 2.296 | Acc: 18.438% (236/1280)
Loss: 2.287 | Acc: 18.229% (245/1344)
Loss: 2.278 | Acc: 18.182% (256/1408)
Loss: 2.274 | Acc: 18.614% (274/1472)
Loss: 2.267 | Acc: 18.555% (285/1536)
Loss: 2.251 | Acc: 19.125% (306/1600)
Loss: 2.246 | Acc: 19.231% (320/1664)
Loss: 2.237 | Acc: 19.618% (339/1728)
Loss

Loss: 1.850 | Acc: 32.138% (4381/13632)
Loss: 1.849 | Acc: 32.170% (4406/13696)
Loss: 1.848 | Acc: 32.209% (4432/13760)
Loss: 1.846 | Acc: 32.219% (4454/13824)
Loss: 1.846 | Acc: 32.301% (4486/13888)
Loss: 1.844 | Acc: 32.382% (4518/13952)
Loss: 1.844 | Acc: 32.420% (4544/14016)
Loss: 1.842 | Acc: 32.450% (4569/14080)
Loss: 1.840 | Acc: 32.523% (4600/14144)
Loss: 1.839 | Acc: 32.594% (4631/14208)
Loss: 1.838 | Acc: 32.644% (4659/14272)
Loss: 1.837 | Acc: 32.694% (4687/14336)
Loss: 1.836 | Acc: 32.743% (4715/14400)
Loss: 1.836 | Acc: 32.750% (4737/14464)
Loss: 1.835 | Acc: 32.785% (4763/14528)
Loss: 1.834 | Acc: 32.812% (4788/14592)
Loss: 1.832 | Acc: 32.874% (4818/14656)
Loss: 1.831 | Acc: 32.914% (4845/14720)
Loss: 1.831 | Acc: 32.927% (4868/14784)
Loss: 1.829 | Acc: 32.947% (4892/14848)
Loss: 1.828 | Acc: 32.973% (4917/14912)
Loss: 1.827 | Acc: 33.006% (4943/14976)
Loss: 1.825 | Acc: 33.098% (4978/15040)
Loss: 1.825 | Acc: 33.110% (5001/15104)
Loss: 1.824 | Acc: 33.155% (5029/15168)


Loss: 1.721 | Acc: 37.025% (9905/26752)
Loss: 1.720 | Acc: 37.079% (9943/26816)
Loss: 1.720 | Acc: 37.083% (9968/26880)
Loss: 1.720 | Acc: 37.066% (9987/26944)
Loss: 1.719 | Acc: 37.074% (10013/27008)
Loss: 1.719 | Acc: 37.075% (10037/27072)
Loss: 1.718 | Acc: 37.106% (10069/27136)
Loss: 1.718 | Acc: 37.125% (10098/27200)
Loss: 1.718 | Acc: 37.122% (10121/27264)
Loss: 1.717 | Acc: 37.149% (10152/27328)
Loss: 1.716 | Acc: 37.190% (10187/27392)
Loss: 1.716 | Acc: 37.194% (10212/27456)
Loss: 1.716 | Acc: 37.206% (10239/27520)
Loss: 1.716 | Acc: 37.214% (10265/27584)
Loss: 1.715 | Acc: 37.218% (10290/27648)
Loss: 1.715 | Acc: 37.211% (10312/27712)
Loss: 1.714 | Acc: 37.226% (10340/27776)
Loss: 1.713 | Acc: 37.259% (10373/27840)
Loss: 1.713 | Acc: 37.278% (10402/27904)
Loss: 1.713 | Acc: 37.289% (10429/27968)
Loss: 1.712 | Acc: 37.304% (10457/28032)
Loss: 1.712 | Acc: 37.311% (10483/28096)
Loss: 1.711 | Acc: 37.330% (10512/28160)
Loss: 1.711 | Acc: 37.376% (10549/28224)
Loss: 1.710 | Acc: 3

...

Loss: 1.373 | Acc: 51.048% (5064/9920)
Loss: 1.374 | Acc: 51.032% (5095/9984)
Loss: 1.374 | Acc: 51.010% (5101/10000)
Evaluation of Epoch 1 is completed, Test accuracy for this epoch is 51.01

Epoch: 2
Loss: 1.286 | Acc: 45.312% (29/64)
Loss: 1.334 | Acc: 49.219% (63/128)
Loss: 1.243 | Acc: 52.083% (100/192)
Loss: 1.273 | Acc: 52.344% (134/256)
Loss: 1.259 | Acc: 53.438% (171/320)
Loss: 1.261 | Acc: 54.427% (209/384)
Loss: 1.278 | Acc: 52.679% (236/448)
Loss: 1.278 | Acc: 52.539% (269/512)
Loss: 1.249 | Acc: 53.819% (310/576)
Loss: 1.239 | Acc: 54.219% (347/640)
Loss: 1.267 | Acc: 53.409% (376/704)
Loss: 1.278 | Acc: 52.995% (407/768)
Loss: 1.258 | Acc: 54.207% (451/832)
Loss: 1.249 | Acc: 54.129% (485/896)
Loss: 1.257 | Acc: 53.958% (518/960)
Loss: 1.255 | Acc: 53.906% (552/1024)
Loss: 1.254 | Acc: 54.044% (588/1088)
Loss: 1.252 | Acc: 54.427% (627/1152)
Loss: 1.255 | Acc: 54.359% (661/1216)
Loss: 1.252 | Acc: 54.375% (696/1280)
Loss: 1.241 | Acc: 54.539% (733/1344)
Loss: 1.250 | Acc:

Loss: 1.236 | Acc: 55.461% (7312/13184)
Loss: 1.237 | Acc: 55.450% (7346/13248)
Loss: 1.238 | Acc: 55.454% (7382/13312)
Loss: 1.239 | Acc: 55.450% (7417/13376)
Loss: 1.239 | Acc: 55.424% (7449/13440)
Loss: 1.239 | Acc: 55.406% (7482/13504)
Loss: 1.239 | Acc: 55.380% (7514/13568)
Loss: 1.240 | Acc: 55.370% (7548/13632)
Loss: 1.240 | Acc: 55.359% (7582/13696)
Loss: 1.241 | Acc: 55.334% (7614/13760)
Loss: 1.240 | Acc: 55.353% (7652/13824)
Loss: 1.241 | Acc: 55.321% (7683/13888)
Loss: 1.239 | Acc: 55.340% (7721/13952)
Loss: 1.239 | Acc: 55.337% (7756/14016)
Loss: 1.238 | Acc: 55.384% (7798/14080)
Loss: 1.237 | Acc: 55.395% (7835/14144)
Loss: 1.237 | Acc: 55.335% (7862/14208)
Loss: 1.238 | Acc: 55.290% (7891/14272)
Loss: 1.237 | Acc: 55.357% (7936/14336)
Loss: 1.236 | Acc: 55.354% (7971/14400)
Loss: 1.236 | Acc: 55.365% (8008/14464)
Loss: 1.234 | Acc: 55.424% (8052/14528)
Loss: 1.234 | Acc: 55.400% (8084/14592)
Loss: 1.235 | Acc: 55.349% (8112/14656)
Loss: 1.235 | Acc: 55.367% (8150/14720)


Loss: 1.236 | Acc: 55.132% (14396/26112)
Loss: 1.236 | Acc: 55.154% (14437/26176)
Loss: 1.236 | Acc: 55.152% (14472/26240)
Loss: 1.236 | Acc: 55.167% (14511/26304)
Loss: 1.236 | Acc: 55.169% (14547/26368)
Loss: 1.236 | Acc: 55.160% (14580/26432)
Loss: 1.236 | Acc: 55.163% (14616/26496)
Loss: 1.235 | Acc: 55.177% (14655/26560)
Loss: 1.235 | Acc: 55.191% (14694/26624)
Loss: 1.235 | Acc: 55.197% (14731/26688)
Loss: 1.235 | Acc: 55.211% (14770/26752)
Loss: 1.235 | Acc: 55.195% (14801/26816)
Loss: 1.235 | Acc: 55.205% (14839/26880)
Loss: 1.235 | Acc: 55.211% (14876/26944)
Loss: 1.235 | Acc: 55.206% (14910/27008)
Loss: 1.235 | Acc: 55.212% (14947/27072)
Loss: 1.235 | Acc: 55.226% (14986/27136)
Loss: 1.235 | Acc: 55.228% (15022/27200)
Loss: 1.235 | Acc: 55.223% (15056/27264)
Loss: 1.235 | Acc: 55.233% (15094/27328)
Loss: 1.234 | Acc: 55.250% (15134/27392)
Loss: 1.234 | Acc: 55.252% (15170/27456)
Loss: 1.234 | Acc: 55.262% (15208/27520)
Loss: 1.234 | Acc: 55.271% (15246/27584)
Loss: 1.234 | Ac

Loss: 1.236 | Acc: 55.253% (21500/38912)
Loss: 1.236 | Acc: 55.247% (21533/38976)
Loss: 1.237 | Acc: 55.233% (21563/39040)
Loss: 1.236 | Acc: 55.237% (21600/39104)
Loss: 1.236 | Acc: 55.234% (21634/39168)
Loss: 1.236 | Acc: 55.238% (21671/39232)
Loss: 1.237 | Acc: 55.242% (21708/39296)
Loss: 1.236 | Acc: 55.249% (21746/39360)
Loss: 1.236 | Acc: 55.240% (21778/39424)
Loss: 1.237 | Acc: 55.219% (21805/39488)
Loss: 1.237 | Acc: 55.216% (21839/39552)
Loss: 1.237 | Acc: 55.233% (21881/39616)
Loss: 1.237 | Acc: 55.242% (21920/39680)
Loss: 1.236 | Acc: 55.256% (21961/39744)
Loss: 1.236 | Acc: 55.275% (22004/39808)
Loss: 1.236 | Acc: 55.282% (22042/39872)
Loss: 1.236 | Acc: 55.278% (22076/39936)
Loss: 1.236 | Acc: 55.265% (22106/40000)
Loss: 1.237 | Acc: 55.249% (22135/40064)
Loss: 1.237 | Acc: 55.238% (22166/40128)
Loss: 1.237 | Acc: 55.232% (22199/40192)
Loss: 1.238 | Acc: 55.222% (22230/40256)
Loss: 1.238 | Acc: 55.221% (22265/40320)
Loss: 1.237 | Acc: 55.237% (22307/40384)
Loss: 1.237 | Ac

Loss: 1.281 | Acc: 54.119% (1524/2816)
Loss: 1.282 | Acc: 54.097% (1558/2880)
Loss: 1.279 | Acc: 54.314% (1599/2944)
Loss: 1.277 | Acc: 54.355% (1635/3008)
Loss: 1.279 | Acc: 54.297% (1668/3072)
Loss: 1.280 | Acc: 54.082% (1696/3136)
Loss: 1.281 | Acc: 54.062% (1730/3200)
Loss: 1.280 | Acc: 54.013% (1763/3264)
Loss: 1.281 | Acc: 53.996% (1797/3328)
Loss: 1.281 | Acc: 53.980% (1831/3392)
Loss: 1.283 | Acc: 53.993% (1866/3456)
Loss: 1.283 | Acc: 53.949% (1899/3520)
Loss: 1.281 | Acc: 53.990% (1935/3584)
Loss: 1.284 | Acc: 53.975% (1969/3648)
Loss: 1.281 | Acc: 54.095% (2008/3712)
Loss: 1.282 | Acc: 54.105% (2043/3776)
Loss: 1.278 | Acc: 54.115% (2078/3840)
Loss: 1.278 | Acc: 54.175% (2115/3904)
Loss: 1.278 | Acc: 54.083% (2146/3968)
Loss: 1.276 | Acc: 54.117% (2182/4032)
Loss: 1.278 | Acc: 54.053% (2214/4096)
Loss: 1.280 | Acc: 53.966% (2245/4160)
Loss: 1.279 | Acc: 54.048% (2283/4224)
Loss: 1.279 | Acc: 54.128% (2321/4288)
Loss: 1.279 | Acc: 54.113% (2355/4352)
Loss: 1.276 | Acc: 54.325